# İş Problemi

İngiltere merkezli perakende şirketi satış ve pazarlama faaliyetleri için roadmap belirlemek istemektedir. Şirketin orta uzun vadeli plan yapabilmesi için var olan müşterilerin gelecekte şirkete sağlayacakları potansiyel değerin tahmin edilmesi gerekmektedir.

# Veri Seti Hikayesi

Online Retail II isimli veri seti İngiltere merkezli bir perakende şirketinin 01/12/2009 - 09/12/2011 tarihleri arasındaki online satış işlemlerini içeriyor. Şirketin ürün kataloğunda hediyelik eşyalar yer almaktadır ve çoğu müşterisinin toptancı olduğu bilgisi mevcuttur.

# Görev 1: BG-NBD ve Gamma-Gamma Modellerini Kurarak 6 Aylık CLTV Tahmini Yapılması

Adım 1: 2010-2011 yıllarındaki veriyi kullanarak İngiltere’deki müşteriler için 6 aylık CLTV tahmini yapınız.

Adım 2: Elde ettiğiniz sonuçları yorumlayıp, değerlendiriniz.

In [172]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.float_format', lambda x: '%.4f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

df_ = pd.read_excel('/Users/mericgenc/Desktop/Python - Data Science/Miuul Data Scientist Path/02 - CRM Analytics/00_datasets/online_retail_II.xlsx', sheet_name='Year 2010-2011')

In [222]:
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom


In [223]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [224]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541910.0000,9.5522,218.0810,-80995.0000,1.0000,3.0000,10.0000,80995.0000
Price,541910.0000,4.6111,96.7598,-11062.0600,1.2500,2.0800,4.1300,38970.0000
Customer ID,406830.0000,15287.6842,1713.6031,12346.0000,13953.0000,15152.0000,16791.0000,18287.0000


## Veri Ön İşleme

In [225]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = round(low_limit,0)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = round(up_limit,0)

In [226]:
df.dropna(inplace=True)
df = df[~df["Invoice"].str.contains("C", na=False)]
df = df[df["Quantity"] > 0]
df = df[df["Price"] > 0]

In [227]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397885.0000,12.9882,179.3316,1.0000,2.0000,6.0000,12.0000,80995.0000
Price,397885.0000,3.1165,22.0979,0.0010,1.2500,1.9500,3.7500,8142.7500
Customer ID,397885.0000,15294.4169,1713.1444,12346.0000,13969.0000,15159.0000,16795.0000,18287.0000


In [228]:
replace_with_thresholds(df, 'Quantity')
replace_with_thresholds(df, 'Price')

In [229]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [230]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397885.0000,11.8296,25.5095,1.0000,2.0000,6.0000,12.0000,298.0000
Price,397885.0000,2.8934,3.2261,0.0010,1.2500,1.9500,3.7500,37.0000
Customer ID,397885.0000,15294.4169,1713.1444,12346.0000,13969.0000,15159.0000,16795.0000,18287.0000


In [231]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom


In [232]:
df['TotalPrice'] = df['Quantity'] + df['Price']
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom,8.5500
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom,9.3900
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom,10.7500
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom,9.3900
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom,9.3900


In [233]:
analysis_date = dt.datetime(2011, 12, 11)

## Customer Lifetime Verisinin Hazırlanması

recency: Son satın alma üzerinden geçen zaman. Haftalık. (kullanıcı özelinde)

T: Müşterinin yaşı. Haftalık. (analiz tarihinden ne kadar süre önce ilk satın alma yapılmış)

frequency: tekrar eden toplam satın alma sayısı (frequency>1)

monetary: satın alma başına ortalama kazanç


In [234]:
clv_df = df.groupby('Customer ID').agg({
    'InvoiceDate': [lambda date: (date.max() - date.min()).days,
                    lambda date: (analysis_date - date.min()).days],
    'Invoice': lambda invoice: invoice.nunique(),
    'TotalPrice': lambda price: price.sum()
})
clv_df.head()

InvoiceDate             Invoice TotalPrice
             <lambda_0> <lambda_1> <lambda>   <lambda>
Customer ID                                           
12346.0000            0        326        1   299.0400
12347.0000          365        368        7  2939.2100
12348.0000          282        359        4  2507.7100
12349.0000            0         19        1   970.1500
12350.0000            0        311        1   259.3000

In [235]:
clv_df.columns = clv_df.columns.droplevel(0)
clv_df.columns = ['recency', 'T', 'frequency', 'monetary']
# Monetary için satınalma başına ortalama kazancı bulmamız gerektiğinden frequency'ye bölmemiz gerekiyor;
clv_df["monetary"] = clv_df["monetary"] / clv_df["frequency"]
clv_df.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4338.0000,130.4486,132.0396,0.0000,0.0000,92.5000,251.7500,373.0000
T,4338.0000,223.8310,117.8546,1.0000,113.0000,249.0000,327.0000,374.0000
frequency,4338.0000,4.2720,7.6980,1.0000,1.0000,2.0000,5.0000,209.0000
monetary,4338.0000,280.5754,252.4247,3.4750,140.7221,226.1040,341.0956,4544.3500


In [236]:
clv_df

,recency,T,frequency,monetary
Customer ID,,,,
12346.0000,0,326,1,299.0400
12347.0000,365,368,7,419.8871
12348.0000,282,359,4,626.9275
12349.0000,0,19,1,970.1500
12350.0000,0,311,1,259.3000
...,...,...,...,...
18280.0000,0,278,1,92.6500
18281.0000,0,181,1,93.3600
18282.0000,118,127,2,82.6950


In [237]:
clv_df = clv_df[(clv_df['frequency'] > 1)]
clv_df["recency"] = clv_df["recency"] / 7
clv_df["T"] = clv_df["T"] / 7

In [238]:
clv_df.head()

,recency,T,frequency,monetary
Customer ID,,,,
12347.0000,52.1429,52.5714,7,419.8871
12348.0000,40.2857,51.2857,4,626.9275
12352.0000,37.1429,42.4286,8,143.2263
12356.0000,43.1429,46.5714,3,593.2900
12358.0000,21.2857,21.5714,2,199.6050


## BG-NBD Modelinin Kurulması

In [239]:
bgf = BetaGeoFitter(penalizer_coef=0.001)

bgf.fit(clv_df['frequency'],
        clv_df['recency'],
        clv_df['T'])

<lifetimes.BetaGeoFitter: fitted with 2845 subjects, a: 0.12, alpha: 11.41, b: 2.49, r: 2.18>

In [240]:
# 6 ay içinde en çok satın alma beklediğimiz 10 müşteri kimdir?
################################################################

bgf.conditional_expected_number_of_purchases_up_to_time(4*6,
                                                        clv_df['frequency'],
                                                        clv_df['recency'],
                                                        clv_df['T']).sort_values(ascending=False).head(10)

Customer ID
12748.0000   76.5418
14911.0000   73.6417
17841.0000   45.6983
13089.0000   36.2063
14606.0000   34.4798
15311.0000   33.7665
12971.0000   31.9589
14646.0000   28.3924
13408.0000   23.2269
18102.0000   22.8061
dtype: float64

In [241]:
clv_df['expected_purchase_6_months'] = bgf.conditional_expected_number_of_purchases_up_to_time(4*6,
                                                        clv_df['frequency'],
                                                        clv_df['recency'],
                                                        clv_df['T'])

In [242]:
clv_df.head()

,recency,T,frequency,monetary,expected_purchase_6_months
Customer ID,,,,,
12347.0000,52.1429,52.5714,7,419.8871,3.3239
12348.0000,40.2857,51.2857,4,626.9275,2.1618
12352.0000,37.1429,42.4286,8,143.2263,4.2775
12356.0000,43.1429,46.5714,3,593.2900,2.0219
12358.0000,21.2857,21.5714,2,199.6050,2.8279


## GAMMA-GAMMA Modelinin Kurulması

In [243]:
ggf = GammaGammaFitter(penalizer_coef=0.01)

ggf.fit(clv_df['frequency'], clv_df['monetary'])

<lifetimes.GammaGammaFitter: fitted with 2845 subjects, p: 3.88, q: 0.37, v: 3.81>

In [244]:
ggf.conditional_expected_average_profit(clv_df['frequency'],
                                        clv_df['monetary']).sort_values(ascending=False).head(10)

Customer ID
12415.0000   3538.3065
14646.0000   2613.1722
12590.0000   2292.8757
14096.0000   2274.7345
17511.0000   2137.1841
16984.0000   2095.0839
12409.0000   2094.0292
12753.0000   2035.5839
13081.0000   1976.9074
17404.0000   1897.8152
dtype: float64

In [245]:
clv_df["expected_average_profit"] = ggf.conditional_expected_average_profit(clv_df['frequency'],
                                                                             clv_df['monetary'])

In [246]:
clv_df.sort_values("expected_average_profit", ascending=False).head(10)

,recency,T,frequency,monetary,expected_purchase_6_months,expected_average_profit
Customer ID,,,,,,
12415.0000,44.7143,48.2857,21,3510.9562,8.9251,3538.3065
14646.0000,50.4286,50.7143,73,2607.3478,28.3924,2613.1722
12590.0000,0.0000,30.2857,2,2106.1100,0.2687,2292.8757
14096.0000,13.8571,14.5714,17,2252.9341,16.7786,2274.7345
17511.0000,52.8571,53.4286,31,2125.9445,11.9822,2137.1841
16984.0000,5.8571,18.7143,2,1924.2650,2.3701,2095.0839
12409.0000,14.7143,26.1429,3,1980.2067,2.7281,2094.0292
12753.0000,48.4286,51.8571,6,1980.2433,2.9648,2035.5839
13081.0000,51.2857,53.1429,11,1947.5818,4.7361,1976.9074


## BG-NBD ve GG modeli ile CLTV'nin hesaplanması

In [247]:
clv_df['clv_6M'] = ggf.customer_lifetime_value(bgf,
                                   clv_df['frequency'],
                                   clv_df['recency'],
                                   clv_df['T'],
                                   clv_df['monetary'],
                                   time=6,  # 6 aylık
                                   freq="W",  # T'nin frekans bilgisi.
                                   discount_rate=0.01)

clv_df.head()                                

,recency,T,frequency,monetary,expected_purchase_6_months,expected_average_profit,clv_6M
Customer ID,,,,,,,
12347.0000,52.1429,52.5714,7,419.8871,3.3239,430.3443,1498.7416
12348.0000,40.2857,51.2857,4,626.9275,2.1618,654.2538,1481.8709
12352.0000,37.1429,42.4286,8,143.2263,4.2775,146.6582,657.1729
12356.0000,43.1429,46.5714,3,593.2900,2.0219,628.3317,1330.8664
12358.0000,21.2857,21.5714,2,199.6050,2.8279,219.1799,648.8122


In [248]:
clv_df = clv_df.reset_index()
clv_df.sort_values(by="clv_6M", ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_purchase_6_months,expected_average_profit,clv_6M
1122,14646.0000,50.4286,50.7143,73,2607.3478,28.3924,2613.1722,77740.1381
843,14096.0000,13.8571,14.5714,17,2252.9341,16.7786,2274.7345,39942.9755
1257,14911.0000,53.1429,53.4286,201,504.4106,73.6417,504.8345,38955.5931
36,12415.0000,44.7143,48.2857,21,3510.9562,8.9251,3538.3065,33087.0010
2487,17511.0000,52.8571,53.4286,31,2125.9445,11.9822,2137.1841,26833.0794
949,14298.0000,50.2857,51.5714,44,1361.5725,17.1291,1366.6675,24528.8024
874,14156.0000,51.5714,53.1429,55,1098.8669,20.6651,1102.1675,23865.8266
650,13694.0000,52.7143,53.4286,50,1157.5638,18.8601,1161.3854,22951.6207
2761,18102.0000,52.2857,52.5714,60,951.3858,22.8061,954.0132,22797.8559
2458,17450.0000,51.2857,52.5714,46,998.6591,17.5988,1002.2552,18481.9439


In [259]:
clv_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Customer ID,2845.0000,15292.1656,1718.6493,12347.0000,13817.0000,15265.0000,16792.0000,18287.0000
recency,2845.0000,28.4151,16.2667,0.0000,14.0000,29.5714,42.7143,53.2857
T,2845.0000,36.9692,14.7130,0.4286,27.4286,40.4286,51.2857,53.4286
frequency,2845.0000,5.9891,9.0442,2.0000,2.0000,4.0000,6.0000,209.0000
monetary,2845.0000,282.5950,225.4709,3.4750,150.7350,234.2517,346.2305,3510.9562
expected_purchase_6_months,2845.0000,3.6530,3.4807,0.0000,1.9729,2.9812,4.4282,76.5418
expected_average_profit,2845.0000,297.3602,234.7603,5.8503,159.5253,245.9955,364.5123,3538.3065
clv_6M,2845.0000,1229.8636,2487.8859,0.0000,404.6402,774.0060,1396.1079,77740.1381
clv_1M,2845.0000,215.7607,434.2820,0.0000,70.9976,135.6862,244.2350,13519.3771
clv_12M,2845.0000,2333.9225,4737.8742,0.0000,769.2257,1470.1094,2646.0912,148394.9806


In [260]:
clv_df['clv_6M'].sum()

3498961.8522780077

Elde ettiğimiz Customer Lifetime Value müşterilerimizin 6 aylık periyotta bize ne kadar kazandıracağını tahminliyor.
Hesabımıza göre bu müşteriler bize toplamda 3.5m birim gelir sağlayacak.

# Görev 2: Farklı Zaman Periyotlarından Oluşan CLTV Analizi

Adım 1: 2010-2011 UK müşterileri için 1 aylık ve 12 aylık CLTV hesaplayınız.

Adım 2: 1 aylık CLTV'de en yüksek olan 10 kişi ile 12 aylık'taki en yüksek 10 kişiyi analiz ediniz.

Adım 3: Fark var mı? Varsa sizce neden olabilir?

Not: Sıfırdan model kurulmasına gerek yoktur. Önceki görevde oluşturulan model üzerinden ilerlenebilir.

In [249]:
clv_df['clv_1M'] = ggf.customer_lifetime_value(bgf,
                                   clv_df['frequency'],
                                   clv_df['recency'],
                                   clv_df['T'],
                                   clv_df['monetary'],
                                   time=1,  # 1 aylık
                                   freq="W",  # T'nin frekans bilgisi.
                                   discount_rate=0.01)

clv_df['clv_12M'] = ggf.customer_lifetime_value(bgf,
                                   clv_df['frequency'],
                                   clv_df['recency'],
                                   clv_df['T'],
                                   clv_df['monetary'],
                                   time=12,  # 12 aylık
                                   freq="W",  # T'nin frekans bilgisi.
                                   discount_rate=0.01)

clv_df.head()                                   


,Customer ID,recency,T,frequency,monetary,expected_purchase_6_months,expected_average_profit,clv_6M,clv_1M,clv_12M
0,12347.0000,52.1429,52.5714,7,419.8871,3.3239,430.3443,1498.7416,260.7260,2860.0818
1,12348.0000,40.2857,51.2857,4,626.9275,2.1618,654.2538,1481.8709,257.9815,2826.1901
2,12352.0000,37.1429,42.4286,8,143.2263,4.2775,146.6582,657.1729,114.6427,1251.3715
3,12356.0000,43.1429,46.5714,3,593.2900,2.0219,628.3317,1330.8664,232.0542,2535.0845
4,12358.0000,21.2857,21.5714,2,199.6050,2.8279,219.1799,648.8122,114.5438,1225.1781


In [250]:
clv_df.sort_values('clv_1M', ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_purchase_6_months,expected_average_profit,clv_6M,clv_1M,clv_12M
1122,14646.0000,50.4286,50.7143,73,2607.3478,28.3924,2613.1722,77740.1381,13519.3771,148394.9806
843,14096.0000,13.8571,14.5714,17,2252.9341,16.7786,2274.7345,39942.9755,7078.6799,75254.9052
1257,14911.0000,53.1429,53.4286,201,504.4106,73.6417,504.8345,38955.5931,6769.6911,74404.1542
36,12415.0000,44.7143,48.2857,21,3510.9562,8.9251,3538.3065,33087.0010,5759.0965,63113.4928
2487,17511.0000,52.8571,53.4286,31,2125.9445,11.9822,2137.1841,26833.0794,4664.0313,51241.5893
949,14298.0000,50.2857,51.5714,44,1361.5725,17.1291,1366.6675,24528.8024,4265.0843,46827.2465
874,14156.0000,51.5714,53.1429,55,1098.8669,20.6651,1102.1675,23865.8266,4148.1098,45576.6912
650,13694.0000,52.7143,53.4286,50,1157.5638,18.8601,1161.3854,22951.6207,3989.0043,43832.6836
2761,18102.0000,52.2857,52.5714,60,951.3858,22.8061,954.0132,22797.8559,3962.9692,43532.8699
2458,17450.0000,51.2857,52.5714,46,998.6591,17.5988,1002.2552,18481.9439,3212.8580,35290.4265


In [251]:
clv_df.sort_values('clv_12M', ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_purchase_6_months,expected_average_profit,clv_6M,clv_1M,clv_12M
1122,14646.0000,50.4286,50.7143,73,2607.3478,28.3924,2613.1722,77740.1381,13519.3771,148394.9806
843,14096.0000,13.8571,14.5714,17,2252.9341,16.7786,2274.7345,39942.9755,7078.6799,75254.9052
1257,14911.0000,53.1429,53.4286,201,504.4106,73.6417,504.8345,38955.5931,6769.6911,74404.1542
36,12415.0000,44.7143,48.2857,21,3510.9562,8.9251,3538.3065,33087.0010,5759.0965,63113.4928
2487,17511.0000,52.8571,53.4286,31,2125.9445,11.9822,2137.1841,26833.0794,4664.0313,51241.5893
949,14298.0000,50.2857,51.5714,44,1361.5725,17.1291,1366.6675,24528.8024,4265.0843,46827.2465
874,14156.0000,51.5714,53.1429,55,1098.8669,20.6651,1102.1675,23865.8266,4148.1098,45576.6912
650,13694.0000,52.7143,53.4286,50,1157.5638,18.8601,1161.3854,22951.6207,3989.0043,43832.6836
2761,18102.0000,52.2857,52.5714,60,951.3858,22.8061,954.0132,22797.8559,3962.9692,43532.8699
2458,17450.0000,51.2857,52.5714,46,998.6591,17.5988,1002.2552,18481.9439,3212.8580,35290.4265


# Görev 3: Segmentasyon ve Aksiyon Önerileri

Adım 1: 2010-2011 UK müşterileri için 6 aylık CLTV'ye göre tüm müşterilerinizi 4 gruba (segmente) ayırınız ve grup isimlerini veri setine
ekleyiniz.

Adım 2: 4 grup içerisinden seçeceğiniz 2 grup için yönetime kısa kısa 6 aylık aksiyon önerilerinde bulununuz.

In [252]:
clv_df["segment"] = pd.qcut(clv_df["clv_6M"], 4, labels=["D", "C", "B", "A"])

clv_df.sort_values(by="clv_6M", ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_purchase_6_months,expected_average_profit,clv_6M,clv_1M,clv_12M,segment
1122,14646.0000,50.4286,50.7143,73,2607.3478,28.3924,2613.1722,77740.1381,13519.3771,148394.9806,A
843,14096.0000,13.8571,14.5714,17,2252.9341,16.7786,2274.7345,39942.9755,7078.6799,75254.9052,A
1257,14911.0000,53.1429,53.4286,201,504.4106,73.6417,504.8345,38955.5931,6769.6911,74404.1542,A
36,12415.0000,44.7143,48.2857,21,3510.9562,8.9251,3538.3065,33087.0010,5759.0965,63113.4928,A
2487,17511.0000,52.8571,53.4286,31,2125.9445,11.9822,2137.1841,26833.0794,4664.0313,51241.5893,A
949,14298.0000,50.2857,51.5714,44,1361.5725,17.1291,1366.6675,24528.8024,4265.0843,46827.2465,A
874,14156.0000,51.5714,53.1429,55,1098.8669,20.6651,1102.1675,23865.8266,4148.1098,45576.6912,A
650,13694.0000,52.7143,53.4286,50,1157.5638,18.8601,1161.3854,22951.6207,3989.0043,43832.6836,A
2761,18102.0000,52.2857,52.5714,60,951.3858,22.8061,954.0132,22797.8559,3962.9692,43532.8699,A
2458,17450.0000,51.2857,52.5714,46,998.6591,17.5988,1002.2552,18481.9439,3212.8580,35290.4265,A


In [257]:
clv_df[['recency', 'T', 'frequency', 'monetary', 'expected_purchase_6_months', 'expected_average_profit', 'clv_1M', 'clv_6M','clv_12M','segment']].groupby("segment").agg(
    {"count", "mean", "sum"})

recency                        T                    frequency               monetary                      expected_purchase_6_months                  expected_average_profit                      clv_1M                      clv_6M                        clv_12M                       
          count        sum    mean count        sum    mean     count   sum    mean    count         sum     mean                      count       sum   mean                   count         sum     mean  count         sum     mean  count          sum      mean   count          sum      mean
segment                                                                                                                                                                                                                                                                                            
D           712 15663.0000 21.9986   712 28671.2857 40.2687       712  2181  3.0632      712  96300.4227 135.2534                        712 1188.1064 1.6687                     712 104105.5735 146.2157    712  26367.6999  37.0333    712  150657.7443  211.5980     712  286250.2739  402.0369
C           711 21798.7143 30.6592   711 27246.4286 38.3213       711  2978  4.1885      711 157016.6535 220.8392                        711 2020.3421 2.8416                     711 167147.5734 235.0880    711  73236.0668 103.0043    711  418133.8861  588.0927     711  794118.7888 1116.9041
B           711 21498.2857 30.2367   711 25296.2857 35.5785       711  3989  5.6104      711 209510.7962 294.6706                        711 2683.8832 3.7748                     711 220276.3966 309.8121    711 132092.6994 185.7844    711  752910.1947 1058.9454     711 1428509.9654 2009.1561
A           711 21880.8571 30.7748   711 23963.2857 33.7036       711  7891 11.0985      711 341154.8671 479.8240                        711 4500.5506 6.3299                     711 354460.0978 498.5374    711 382142.6682 537.4721    711 2177260.0271 3062.2504     711 4131130.4930 5810.3101

Sonuçlara baktığımızda segmentler artarken recency'lerin de arttığını görüyoruz, aynı zamanda müşterinin yaşı yani T değeri de düşüyor. Ancak frequencyler ve monetary'ler de artmakta. Yani müşteri yaşı düşse bile frekans ve bırakılan para nedeniyle A segmentine doğru bir skor artışı var. Bu yüzden de beklenen 6 aylık satınalma tutarları da artıyor.

In [262]:
clv_df[clv_df['segment']=='A'].describe().T

,count,mean,std,min,25%,50%,75%,max
Customer ID,711.0000,15029.5767,1760.7863,12347.0000,13448.5000,14895.0000,16608.0000,18287.0000
recency,711.0000,30.7748,18.0069,0.0000,12.7143,34.7143,48.7143,53.2857
T,711.0000,33.7036,17.6689,0.4286,15.7143,38.7143,51.5714,53.4286
frequency,711.0000,11.0985,16.1926,2.0000,3.0000,6.0000,13.0000,209.0000
monetary,711.0000,479.8240,318.4927,88.2094,303.0452,397.1200,536.9121,3510.9562
expected_purchase_6_months,711.0000,6.3299,5.6001,1.4905,3.6808,5.1523,7.0988,76.5418
expected_average_profit,711.0000,498.5374,330.3402,88.7757,309.2651,411.3152,555.9921,3538.3065
clv_6M,711.0000,3062.2504,4460.0716,1397.3512,1709.7438,2098.3296,2850.8385,77740.1381
clv_1M,711.0000,537.4721,777.4271,243.3911,301.6653,366.7462,500.3098,13519.3771
clv_12M,711.0000,5810.3101,8501.5841,2615.4193,3244.1532,3972.0355,5431.8740,148394.9806


Ek olarak A segmentinin %75. değeri ile max değeri arasında büyük bir fark var. Bu segment de kendi içinde ayrılabilir

A segmenti en değerli müşterilerimizden oluşuyor. CLV verilerine göre bize en çok getirisi olan müşteriler. Bu yüzden bu müşterilere ayrıcalıklı davranmamız, buna uygun kampanyalar uygulamamız gerekli.

D segmenti getiri olarak en düşük seviyedeki grubumuz. Bu grubun şirkete getirisini artırmak için ise alışveriş sıklıklarını artırmak ve bununla birlikte getirilerini de artırmak için indirim uygulanabilir.